# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f51d2f32eb0>
├── 'a' --> tensor([[ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082]])
└── 'x' --> <FastTreeValue 0x7f51d2f32df0>
    └── 'c' --> tensor([[ 0.1688,  1.2293, -1.7571,  0.2018],
                        [-1.0934, -0.7801, -0.7857, -0.6936],
                        [-0.7203,  0.0250,  1.1158,  0.2297]])

In [4]:
t.a

tensor([[ 0.5132, -0.0354, -1.3574],
        [ 0.1288, -0.0066, -0.3082]])

In [5]:
%timeit t.a

60.4 ns ± 0.072 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f51d2f32eb0>
├── 'a' --> tensor([[-1.0489, -0.1536,  0.7453],
│                   [ 1.8058,  0.6462,  0.4096]])
└── 'x' --> <FastTreeValue 0x7f51d2f32df0>
    └── 'c' --> tensor([[ 0.1688,  1.2293, -1.7571,  0.2018],
                        [-1.0934, -0.7801, -0.7857, -0.6936],
                        [-0.7203,  0.0250,  1.1158,  0.2297]])

In [7]:
%timeit t.a = new_value

62.9 ns ± 0.0243 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082]]),
    x: Batch(
           c: tensor([[ 0.1688,  1.2293, -1.7571,  0.2018],
                      [-1.0934, -0.7801, -0.7857, -0.6936],
                      [-0.7203,  0.0250,  1.1158,  0.2297]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5132, -0.0354, -1.3574],
        [ 0.1288, -0.0066, -0.3082]])

In [11]:
%timeit b.a

56.1 ns ± 0.0508 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5176, -0.7717,  1.5881],
               [-0.7335,  0.5524, -0.4657]]),
    x: Batch(
           c: tensor([[ 0.1688,  1.2293, -1.7571,  0.2018],
                      [-1.0934, -0.7801, -0.7857, -0.6936],
                      [-0.7203,  0.0250,  1.1158,  0.2297]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.152 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 4.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 85.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f51d0eab8b0>
├── 'a' --> tensor([[[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]],
│           
│                   [[ 0.5132, -0.0354, -1.3574],
│                    [ 0.1288, -0.0066, -0.3082]]])
└── 'x' --> <FastTreeValue 0x7f5119154f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 93.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5119793430>
├── 'a' --> tensor([[ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082],
│                   [ 0.5132, -0.0354, -1.3574],
│                   [ 0.1288, -0.0066, -0.3082]])
└── 'x' --> <FastTreeValue 0x7f51d0e9ae50>
    └── 'c' --> tensor([[ 0.1688,  1.2293, -1.7571,  0.2018],
                        [-1.0934, -0.7801, -0.7857, -0.6936],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 67 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.2 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]],
       
               [[ 0.5132, -0.0354, -1.3574],
                [ 0.1288, -0.0066, -0.3082]]]),
    x: Batch(
           c: tensor([[[ 0.1688,  1.2293, -1.7571,  0.2018],
                       [-1.0934, -0.7801, -0.7857, -0.6936],
                       [-0.7203,  0.0250,  1.1158,  0.2297]],
         

In [26]:
%timeit Batch.stack(batches)

95.3 µs ± 80.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082],
               [ 0.5132, -0.0354, -1.3574],
               [ 0.1288, -0.0066, -0.3082]]),
    x: Batch(
           c: tensor([[ 0.1688,  1.2293, -1.7571,  0.2018],
                      [-1.0934, -0.7801, -0.7857, -0.6936],
                      [-0.7203,  0.0250,  1.1158,  0.2297],
                      [ 0.1688,  1.2293, -1.7571,  0.2018],
                      [-1.0934, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 4.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
